### Idea: Dựa vào sự tương đồng chuỗi/trình tự acid amin

### Sử dụng BLAST/Diamond

## Setup

In [1]:
import os

if not os.path.exists("/usr/bin/diamond"):
    print("Downloading...")
    !wget -q http://github.com/bbuchfink/diamond/releases/download/v2.1.8/diamond-linux64.tar.gz
    !tar xzf diamond-linux64.tar.gz
    
    !mv diamond /usr/bin/
    !chmod +x /usr/bin/diamond
    print("Finished installing!")
else:
    print("Already exists.")

Downloading...
Finished installing!


In [2]:
!diamond --version

diamond version 2.1.8


## Create database

In [3]:
train_fasta = "/workspace/data/Train/train_sequences.fasta"
db_path = "/workspace/data/Train/train_data.dmnd"

cmd = f"diamond makedb --in {train_fasta} -d {db_path} --quiet"

print("Making db...")
!{cmd}
print("Finished")

Making db...
Finished


## Alignment

### Lấy từng protein trong tập Test, đem so với Database vừa tạo
#### --sensitive: Chế độ tìm kiếm kỹ (chậm hơn chút nhưng chính xác hơn).
#### --top 1: Chỉ lấy thằng giống nhất (Best Hit)

#### Cơ chế của diamond: Với tham số --sensitive mặc định, những kết quả mà Diamond trả về thường có độ tương đồng (pident) nằm trong khoảng từ ~20% đến 100% (tức là score từ 0.2 đến 1.0).

#### Khi chạy lệnh --top 1, với mỗi protein trong tập Test, một trong 3 kịch bản sau sẽ xảy ra:

Kịch bản A (Anh em sinh đôi): Tìm thấy protein trong Train giống 90-100%.

Kết quả: Score cao (0.9 - 1.0). Gán nhãn cực chuẩn.

Kịch bản B (Họ hàng xa - Twilight Zone): Tìm thấy protein giống 20-30%.

Kết quả: Score thấp (0.2 - 0.3). Đây là chỗ "nguy hiểm" độ tin cậy rất thấp.

Kịch bản C (Mồ côi - Orphan): Không tìm thấy ai giống cả.

Kết quả: Không có dòng nào trong file output. 

In [5]:
test_fasta = "/workspace/data/Test/testsuperset.fasta"
result_path = "/workspace/notebooks/diamond_results.tsv"

cmd = f"diamond blastp -d {db_path} -q {test_fasta} -o {result_path} --sensitive --top 1 -f 6 qseqid sseqid pident"

print("Aligning...")
!{cmd}
print(f"Finished")

Aligning...
diamond v2.1.8.162 (C) Max Planck Society for the Advancement of Science, Benjamin Buchfink, University of Tuebingen
Documentation, support and updates available at http://www.diamondsearch.org
Please cite: http://dx.doi.org/10.1038/s41592-021-01101-x Nature Methods (2021)

#CPU threads: 32
Scoring parameters: (Matrix=BLOSUM62 Lambda=0.267 K=0.041 Penalties=11/1)
Temporary directory: /workspace/notebooks
Percentage range of top alignment score to report hits: 1
Opening the database...  [0.06s]
Database: /workspace/data/Train/train_data.dmnd (type: Diamond database, sequences: 82404, letters: 43327058)
Block size = 2000000000
Opening the input file...  [0.036s]
Opening the output file...  [0s]
Loading query sequences...  [0.21s]
Masking queries...  [0.14s]
Algorithm: Double-indexed
Building query histograms...  [0.506s]
Seeking in database...  [0s]
Loading reference sequences...  [0.052s]
Masking reference...  [0.061s]
Initializing temporary storage...  [0s]
Building referen

In [7]:
import pandas as pd
diamond_res_df = pd.read_csv('/workspace/notebooks/diamond_results.tsv', sep='\t')

In [8]:
diamond_res_df.head()

,A0A0C5B5G6,sp|A0A0C5B5G6|MOTSC_HUMAN,100
0,A0A1B0GTW7,sp|A0A1L8HYT7|CIROP_XENLA,39.1
1,A0JNW5,sp|A0JNW5|BLT3B_HUMAN,100.0
2,A0JP26,sp|A0JP26|POTB3_HUMAN,100.0
3,A0PK11,sp|A0PK11|CLRN2_HUMAN,100.0
4,A1A4S6,sp|A1A4S6|RHG10_HUMAN,100.0


## Inference

### Lấy nhãn của ProteinTrain_B gán cho ProteinTest_A với điểm số là 0.95.

In [9]:
import numpy as np
from tqdm import tqdm

print("Reading data...")
df_diamond = pd.read_csv(result_path, sep="\t", names=["test_id", "train_id", "pident"])

Reading data...


In [11]:
df_diamond

,test_id,train_id,pident
0,A0A0C5B5G6,sp|A0A0C5B5G6|MOTSC_HUMAN,100.0
1,A0A1B0GTW7,sp|A0A1L8HYT7|CIROP_XENLA,39.1
2,A0JNW5,sp|A0JNW5|BLT3B_HUMAN,100.0
3,A0JP26,sp|A0JP26|POTB3_HUMAN,100.0
4,A0PK11,sp|A0PK11|CLRN2_HUMAN,100.0
...,...,...,...
262525,Q9XFY6,sp|Q9H9S4|CB39L_HUMAN,52.0
262526,Q9Y7I1,sp|Q9Y7I1|YE1K_SCHPO,100.0
262527,Q9Y7P7,sp|Q9Y7P7|YQ63_SCHPO,100.0
262528,Q9Y7Q3,sp|Q9Y7Q3|YQ6A_SCHPO,100.0


In [14]:
train_terms = pd.read_csv("/workspace/data/Train/train_terms.tsv", sep="\t", usecols=["EntryID", "term"])
train_terms_grouped = train_terms.groupby("EntryID")["term"].apply(list).to_dict()

print("Reformatting...")

# Hàm làm sạch ID: biến "sp|A0A0C5B5G6|MOTSC_HUMAN" thành "A0A0C5B5G6"
def clean_id(raw_id):
    if "|" in str(raw_id):
        # Tách chuỗi bằng dấu | và lấy phần tử thứ 2 (index 1)
        return raw_id.split("|")[1]
    return raw_id

# Áp dụng cho cả cột train_id
df_diamond["train_id"] = df_diamond["train_id"].apply(clean_id)

print("Predicting...")
predictions = []

# Duyệt qua từng kết quả so khớp
for _, row in tqdm(df_diamond.iterrows(), total=len(df_diamond)):
    test_id = row['test_id']
    train_match_id = row['train_id']
    score = row['pident'] / 100.0 # Chuyển 95 -> 0.95
    
    # Nếu protein Train khớp đó có nhãn
    if train_match_id in train_terms_grouped:
        associated_terms = train_terms_grouped[train_match_id]
        for term in associated_terms:
            predictions.append(f"{test_id}\t{term}\t{score:.3f}")

# Ghi file submission
print("Đang ghi file submission...")
with open("submission_level2_blast.tsv", "w") as f:
    f.write("\n".join(predictions))

print("Finished")

Reformatting...
Predicting...


100%|██████████| 262530/262530 [00:03<00:00, 84402.44it/s]

Đang ghi file submission...
Finished


### Improvement

#### Có thể Diamond thua Naive approach về độ phủ

#### Hướng cải tiến: ensemble: nếu tìm thấy họ hàng, dùng diamond, o.t lấp chỗ trống bằng Naive

In [15]:
naive_path = "submission_naive.tsv"
blast_path = "submission_level2_blast.tsv"

test_fasta_path = "/workspace/data/Test/testsuperset.fasta"

print("Reading file...")
naive_preds = {}
with open(naive_path, 'r') as f:
    for line in tqdm(f, desc="Reading Naive"):
        parts = line.strip().split('\t')
        if len(parts) < 2: continue
        pid = parts[0]
        if pid not in naive_preds:
            naive_preds[pid] = []
        naive_preds[pid].append(line.strip())

blast_preds = {}
with open(blast_path, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) < 2: continue
        pid = parts[0]
        if pid not in blast_preds:
            blast_preds[pid] = []
        blast_preds[pid].append(line.strip())

from Bio import SeqIO
test_ids = []
with open(test_fasta_path) as handle:
    for record in SeqIO.parse(handle, "fasta"):
        test_ids.append(record.id)
print(f"Need to predict: {len(test_ids)} proteins")

#Hybrid
final_lines = []
count_blast = 0
count_naive = 0

for pid in tqdm(test_ids):
    #DIAMOND
    if pid in blast_preds:
        final_lines.extend(blast_preds[pid])
        count_blast += 1
    #NAIVE
    elif pid in naive_preds:
        final_lines.extend(naive_preds[pid])
        count_naive += 1
        
output_file = "submission_hybrid_v1.tsv"
with open(output_file, "w") as f:
    f.write("\n".join(final_lines))

Reading file...


Reading Naive: 10093905it [00:03, 2753620.09it/s]


Need to predict: 224309 proteins


100%|██████████| 224309/224309 [00:00<00:00, 2537517.49it/s]
